# Ethiopian Medical Data Warehouse: Analytical Dashboard
This notebook provides visualizations for the data collected from Telegram and processed through our YOLO and dbt pipeline. It interacts directly with our **FastAPI** analytical layer.

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set API Base URL (adjust if running on different port)
BASE_URL = "http://localhost:8000"
sns.set_theme(style="whitegrid")

## 1. Visual Content Analysis (YOLO Results)
This visualization shows how different categories of images (Promotional vs. Product Display vs. Lifestyle) perform in terms of user engagement (average views).

In [ ]:
response = requests.get(f"{BASE_URL}/api/reports/visual-content")
if response.status_code == 200:
    data = response.json()
    df_visual = pd.DataFrame(data)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='category', y='avg_views', data=df_visual, palette='viridis')
    plt.title('Average Engagement by Image Category')
    plt.xlabel('Category')
    plt.ylabel('Average Views')
    plt.show()
else:
    print("Error: Ensure the API server is running at http://localhost:8000")

## 2. Top Mentioned Products/Keywords
Analyzing the most frequent keywords in message texts.

In [ ]:
response = requests.get(f"{BASE_URL}/api/reports/top-products?limit=15")
if response.status_code == 200:
    data = response.json()
    df_products = pd.DataFrame(data)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x='frequency', y='keyword', data=df_products, palette='magma')
    plt.title('Top 15 Mentioned Keywords in Medical Channels')
    plt.xlabel('Frequency')
    plt.ylabel('Keyword')
    plt.show()
else:
    print("API Error")

## 3. Channel Activity Comparison
Comparing total posts and average reach across main channels.

In [ ]:
channels = ["CheMed123", "lobelia4cosmetics", "tikvahpharma"]
all_stats = []

for ch in channels:
    res = requests.get(f"{BASE_URL}/api/channels/{ch}/activity")
    if res.status_code == 200:
        all_stats.extend(res.json())

if all_stats:
    df_stats = pd.DataFrame(all_stats)
    
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Bar plot for Post Count
    sns.barplot(x='channel_name', y='post_count', data=df_stats, color='skyblue', ax=ax1, label='Post Count')
    ax1.set_ylabel('Total Posts')
    
    # Twin axis for Avg Views
    ax2 = ax1.twinx()
    sns.lineplot(x='channel_name', y='avg_views', data=df_stats, color='red', marker='o', ax=ax2, label='Avg Views')
    ax2.set_ylabel('Average Views')
    
    plt.title('Comparison of Channel Activity and Engagement')
    plt.show()